# 88. ダウンサンプリング：プーリングとストライド畳み込み

## 学習目標

このノートブックでは、以下を学びます：

1. **なぜダウンサンプリングが必要か**の理解
2. **プーリング**の種類と特性（Max, Average, Global）
3. **ストライド畳み込み**によるダウンサンプリング
4. **プーリング vs ストライド畳み込み**の比較
5. **空間解像度と受容野**のトレードオフ

## 目次

1. [なぜダウンサンプリング？](#section1)
2. [Max Pooling](#section2)
3. [Average Pooling](#section3)
4. [Global Pooling](#section4)
5. [ストライド畳み込み](#section5)
6. [Pooling vs Strided Conv](#section6)
7. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# PyTorchをインポート
import torch
import torch.nn as nn
import torch.nn.functional as F

<a id="section1"></a>
## 1. なぜダウンサンプリング？

### ダウンサンプリングの3つの目的

1. **計算量の削減**：空間サイズを半分にすると、計算量は1/4に
2. **受容野の急速な拡大**：累積ストライドが増加し、深い層で広い範囲を見れる
3. **Translation Invariance（並進不変性）の強化**：位置のずれに対してより頑健に

In [ ]:
def visualize_downsampling_benefits():
    """ダウンサンプリングの利点を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. 計算量の削減
    ax = axes[0]
    sizes = [224, 112, 56, 28, 14, 7]
    ops = [s**2 for s in sizes]  # 相対的な計算量
    ops_normalized = [o / ops[0] * 100 for o in ops]
    
    colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(sizes)))
    bars = ax.bar(range(len(sizes)), ops_normalized, color=colors)
    ax.set_xticks(range(len(sizes)))
    ax.set_xticklabels([f'{s}×{s}' for s in sizes])
    ax.set_ylabel('相対計算量 (%)', fontsize=12)
    ax.set_title('1. 計算量の削減', fontsize=14, fontweight='bold')
    
    for bar, pct in zip(bars, ops_normalized):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
               f'{pct:.1f}%', ha='center', fontsize=10)
    
    # 2. 受容野の拡大
    ax = axes[1]
    # ダウンサンプリングなしとありの比較
    layers = range(8)
    # stride=1のみ
    rf_no_ds = [1 + n * 2 for n in layers]
    # 2層ごとにstride=2
    rf_with_ds = []
    rf, cum_stride = 1, 1
    for n in layers:
        if n == 0:
            rf_with_ds.append(rf)
        else:
            rf = rf + 2 * cum_stride
            rf_with_ds.append(rf)
            if n % 2 == 0:
                cum_stride *= 2
    
    ax.plot(layers, rf_no_ds, 'o-', label='ダウンサンプリングなし', linewidth=2)
    ax.plot(layers, rf_with_ds, 's-', label='ダウンサンプリングあり', linewidth=2)
    ax.set_xlabel('層', fontsize=12)
    ax.set_ylabel('受容野サイズ', fontsize=12)
    ax.set_title('2. 受容野の急速な拡大', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # 3. Translation Invariance
    ax = axes[2]
    # シンプルな図で概念を説明
    # 2x2のmax poolingで位置ずれに対する頑健性を示す
    ax.text(0.5, 0.9, 'Max Poolingによる並進不変性', 
           transform=ax.transAxes, fontsize=14, ha='center', fontweight='bold')
    
    # 入力1（特徴が左上）
    ax.text(0.1, 0.7, '入力1:', transform=ax.transAxes, fontsize=12)
    input1 = np.array([[1, 0], [0, 0]])
    for i in range(2):
        for j in range(2):
            color = 'red' if input1[i, j] == 1 else 'white'
            rect = Rectangle((0.25 + j*0.08, 0.62 - i*0.08), 0.07, 0.07,
                             linewidth=1, edgecolor='black', facecolor=color)
            ax.add_patch(rect)
    
    # 入力2（特徴が右下）
    ax.text(0.1, 0.4, '入力2:', transform=ax.transAxes, fontsize=12)
    input2 = np.array([[0, 0], [0, 1]])
    for i in range(2):
        for j in range(2):
            color = 'red' if input2[i, j] == 1 else 'white'
            rect = Rectangle((0.25 + j*0.08, 0.32 - i*0.08), 0.07, 0.07,
                             linewidth=1, edgecolor='black', facecolor=color)
            ax.add_patch(rect)
    
    # 矢印とMax Pool結果
    ax.annotate('', xy=(0.55, 0.55), xytext=(0.45, 0.55),
               arrowprops=dict(arrowstyle='->', lw=2))
    ax.text(0.5, 0.58, 'Max\nPool', transform=ax.transAxes, fontsize=10, ha='center')
    
    ax.annotate('', xy=(0.55, 0.35), xytext=(0.45, 0.35),
               arrowprops=dict(arrowstyle='->', lw=2))
    ax.text(0.5, 0.38, 'Max\nPool', transform=ax.transAxes, fontsize=10, ha='center')
    
    # 出力（両方とも同じ！）
    ax.text(0.6, 0.55, '出力: 1', transform=ax.transAxes, fontsize=14, color='red')
    ax.text(0.6, 0.35, '出力: 1', transform=ax.transAxes, fontsize=14, color='red')
    
    ax.text(0.5, 0.15, '位置が違っても\n同じ出力！', 
           transform=ax.transAxes, fontsize=12, ha='center',
           bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('3. 並進不変性の強化', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_downsampling_benefits()

<a id="section2"></a>
## 2. Max Pooling

### 定義

Max Poolingは、各ウィンドウ内の**最大値**を取ります：

$$\text{MaxPool}(X)_{i,j} = \max_{(m,n) \in W_{i,j}} X_{m,n}$$

ここで $W_{i,j}$ は位置 $(i,j)$ におけるプーリングウィンドウです。

In [ ]:
def visualize_max_pooling():
    """Max Poolingの動作を可視化"""
    # 入力特徴マップ
    np.random.seed(42)
    input_map = np.random.randint(0, 10, (4, 4))
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 入力
    ax = axes[0]
    im = ax.imshow(input_map, cmap='Blues', vmin=0, vmax=10)
    for i in range(4):
        for j in range(4):
            ax.text(j, i, str(input_map[i, j]), ha='center', va='center', fontsize=16)
    ax.set_title('入力 (4×4)', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 各2x2ウィンドウをハイライト
    for i in range(2):
        for j in range(2):
            rect = Rectangle((j*2-0.5, i*2-0.5), 2, 2,
                             linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
    
    # 操作の説明
    ax = axes[1]
    ax.text(0.5, 0.8, 'Max Pooling 2×2', fontsize=18, ha='center', fontweight='bold',
           transform=ax.transAxes)
    ax.text(0.5, 0.6, 'stride = 2', fontsize=14, ha='center',
           transform=ax.transAxes)
    ax.text(0.5, 0.4, '各2×2ウィンドウから\n最大値を選択', fontsize=14, ha='center',
           transform=ax.transAxes)
    
    # 計算例
    ax.text(0.5, 0.15, f'左上: max({input_map[0,0]},{input_map[0,1]},{input_map[1,0]},{input_map[1,1]}) = {input_map[0:2,0:2].max()}', 
           fontsize=12, ha='center', transform=ax.transAxes)
    
    ax.axis('off')
    
    # 出力
    ax = axes[2]
    # 2x2 max pooling実行
    output_map = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            output_map[i, j] = input_map[i*2:(i+1)*2, j*2:(j+1)*2].max()
    
    im = ax.imshow(output_map, cmap='Blues', vmin=0, vmax=10)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, str(int(output_map[i, j])), ha='center', va='center', fontsize=20)
    ax.set_title('出力 (2×2)', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.suptitle('Max Pooling: 各領域の最大値を抽出', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return input_map, output_map

input_map, output_map = visualize_max_pooling()

In [ ]:
def max_pooling_numpy(x, pool_size=2, stride=None):
    """NumPyでMax Poolingを実装"""
    if stride is None:
        stride = pool_size
    
    h, w = x.shape
    out_h = (h - pool_size) // stride + 1
    out_w = (w - pool_size) // stride + 1
    
    output = np.zeros((out_h, out_w))
    
    for i in range(out_h):
        for j in range(out_w):
            h_start = i * stride
            w_start = j * stride
            window = x[h_start:h_start+pool_size, w_start:w_start+pool_size]
            output[i, j] = np.max(window)
    
    return output

# 検証
print("入力:")
print(input_map)
print("\nNumPy実装の出力:")
print(max_pooling_numpy(input_map, pool_size=2))

# PyTorchと比較
x_torch = torch.tensor(input_map, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
pool = nn.MaxPool2d(2, stride=2)
output_torch = pool(x_torch).squeeze().numpy()
print("\nPyTorchの出力:")
print(output_torch)

### Max Poolingの特性

1. **最も顕著な特徴を保持**：最大活性化を持つ位置の情報を保持
2. **微小な位置変化に頑健**：特徴が少しずれても最大値は同じことが多い
3. **学習パラメータなし**：単純な操作なので高速
4. **勾配の伝播**：最大値の位置のみに勾配が流れる（sparse gradient）

In [ ]:
def visualize_maxpool_gradient():
    """Max Poolingの勾配伝播を可視化"""
    # PyTorchで勾配を計算
    x = torch.tensor([[1., 3., 2., 1.],
                      [4., 6., 5., 2.],
                      [7., 2., 8., 3.],
                      [1., 5., 4., 9.]], requires_grad=True)
    
    # 2x2 max pooling
    x_batch = x.unsqueeze(0).unsqueeze(0)
    pool = nn.MaxPool2d(2, stride=2)
    y = pool(x_batch)
    
    # 全出力の和で逆伝播
    y.sum().backward()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 入力
    ax = axes[0]
    input_np = x.detach().numpy()
    im = ax.imshow(input_np, cmap='Blues', vmin=0, vmax=10)
    for i in range(4):
        for j in range(4):
            ax.text(j, i, f'{input_np[i,j]:.0f}', ha='center', va='center', fontsize=14)
    ax.set_title('入力', fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 出力
    ax = axes[1]
    output_np = y.squeeze().detach().numpy()
    im = ax.imshow(output_np, cmap='Reds', vmin=0, vmax=10)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, f'{output_np[i,j]:.0f}', ha='center', va='center', fontsize=16)
    ax.set_title('MaxPool出力', fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 勾配
    ax = axes[2]
    grad_np = x.grad.numpy()
    im = ax.imshow(grad_np, cmap='Greens', vmin=0, vmax=1)
    for i in range(4):
        for j in range(4):
            ax.text(j, i, f'{grad_np[i,j]:.0f}', ha='center', va='center', fontsize=14,
                   color='white' if grad_np[i,j] > 0.5 else 'black')
    ax.set_title('勾配（∂L/∂x）', fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.suptitle('Max Poolingの勾配：最大値の位置のみに勾配が流れる', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_maxpool_gradient()

<a id="section3"></a>
## 3. Average Pooling

### 定義

Average Poolingは、各ウィンドウ内の**平均値**を取ります：

$$\text{AvgPool}(X)_{i,j} = \frac{1}{|W|} \sum_{(m,n) \in W_{i,j}} X_{m,n}$$

In [ ]:
def visualize_avg_pooling():
    """Average Poolingの動作を可視化"""
    # 同じ入力を使用
    np.random.seed(42)
    input_map = np.random.randint(0, 10, (4, 4)).astype(float)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 入力
    ax = axes[0]
    im = ax.imshow(input_map, cmap='Blues', vmin=0, vmax=10)
    for i in range(4):
        for j in range(4):
            ax.text(j, i, f'{input_map[i, j]:.0f}', ha='center', va='center', fontsize=16)
    ax.set_title('入力 (4×4)', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 操作の説明
    ax = axes[1]
    ax.text(0.5, 0.8, 'Average Pooling 2×2', fontsize=18, ha='center', fontweight='bold',
           transform=ax.transAxes)
    ax.text(0.5, 0.6, 'stride = 2', fontsize=14, ha='center',
           transform=ax.transAxes)
    ax.text(0.5, 0.4, '各2×2ウィンドウの\n平均値を計算', fontsize=14, ha='center',
           transform=ax.transAxes)
    
    # 計算例
    avg_val = input_map[0:2,0:2].mean()
    ax.text(0.5, 0.15, f'左上: ({input_map[0,0]:.0f}+{input_map[0,1]:.0f}+{input_map[1,0]:.0f}+{input_map[1,1]:.0f})/4 = {avg_val:.1f}', 
           fontsize=12, ha='center', transform=ax.transAxes)
    
    ax.axis('off')
    
    # 出力
    ax = axes[2]
    output_map = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            output_map[i, j] = input_map[i*2:(i+1)*2, j*2:(j+1)*2].mean()
    
    im = ax.imshow(output_map, cmap='Blues', vmin=0, vmax=10)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, f'{output_map[i, j]:.1f}', ha='center', va='center', fontsize=18)
    ax.set_title('出力 (2×2)', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.suptitle('Average Pooling: 各領域の平均値を計算', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_avg_pooling()

In [ ]:
def compare_max_vs_avg():
    """Max PoolingとAverage Poolingの比較"""
    # 実際の画像で比較
    from scipy import ndimage
    
    # テスト画像の作成
    np.random.seed(42)
    img = np.zeros((64, 64))
    # いくつかの明るい点を追加
    img[10:15, 10:15] = 1
    img[30:35, 40:45] = 1
    img[50:55, 20:25] = 1
    # ノイズを追加
    img += np.random.normal(0, 0.1, img.shape)
    img = np.clip(img, 0, 1)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 元画像
    axes[0, 0].imshow(img, cmap='gray', vmin=0, vmax=1)
    axes[0, 0].set_title('入力画像 (64×64)', fontsize=12)
    axes[0, 0].axis('off')
    
    # PyTorchでプーリング
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    pool_sizes = [2, 4, 8]
    
    for idx, ps in enumerate(pool_sizes):
        # Max Pooling
        max_pool = nn.MaxPool2d(ps, stride=ps)
        max_out = max_pool(x).squeeze().numpy()
        
        # Average Pooling
        avg_pool = nn.AvgPool2d(ps, stride=ps)
        avg_out = avg_pool(x).squeeze().numpy()
        
        axes[0, idx].imshow(max_out, cmap='gray', vmin=0, vmax=1)
        axes[0, idx].set_title(f'Max Pool {ps}×{ps}\n({max_out.shape[0]}×{max_out.shape[1]})', fontsize=12)
        axes[0, idx].axis('off')
        
        axes[1, idx].imshow(avg_out, cmap='gray', vmin=0, vmax=1)
        axes[1, idx].set_title(f'Avg Pool {ps}×{ps}\n({avg_out.shape[0]}×{avg_out.shape[1]})', fontsize=12)
        axes[1, idx].axis('off')
    
    plt.suptitle('Max Pooling vs Average Pooling の比較', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

compare_max_vs_avg()

### Max vs Average の使い分け

| 特性 | Max Pooling | Average Pooling |
|-----|------------|----------------|
| 保持する情報 | 最も顕著な特徴 | 全体の傾向 |
| ノイズ耐性 | やや低い（ノイズも拾う） | 高い（平均化で抑制） |
| 位置不変性 | 強い | 中程度 |
| 勾配の性質 | sparse（疎） | dense（密） |
| 典型的な用途 | 中間層 | 最終層（GAP） |

<a id="section4"></a>
## 4. Global Pooling

### Global Average Pooling (GAP)

特徴マップ全体を1つの値に集約します：

$$\text{GAP}(X_c) = \frac{1}{H \times W} \sum_{i=1}^{H} \sum_{j=1}^{W} X_{c,i,j}$$

**用途**: 全結合層の代替として、特に画像分類の最終層で使用

In [ ]:
def visualize_global_pooling():
    """Global Average Poolingの動作を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 複数チャンネルの特徴マップ
    np.random.seed(42)
    C, H, W = 3, 7, 7
    feature_maps = np.random.rand(C, H, W)
    
    # 特徴マップを表示
    ax = axes[0]
    # 3チャンネルを縦に並べて表示
    combined = np.vstack([feature_maps[i] for i in range(C)])
    im = ax.imshow(combined, cmap='viridis', vmin=0, vmax=1)
    
    # チャンネル境界
    for i in range(1, C):
        ax.axhline(y=i*H - 0.5, color='white', linewidth=2)
    
    ax.set_title(f'入力特徴マップ\n({C}チャンネル × {H}×{W})', fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([H//2, H + H//2, 2*H + H//2])
    ax.set_yticklabels(['Ch 0', 'Ch 1', 'Ch 2'])
    
    # 操作説明
    ax = axes[1]
    ax.text(0.5, 0.7, 'Global Average\nPooling', fontsize=18, ha='center', fontweight='bold',
           transform=ax.transAxes)
    ax.text(0.5, 0.4, '各チャンネルの\n全ピクセルの平均', fontsize=14, ha='center',
           transform=ax.transAxes)
    ax.text(0.5, 0.15, f'出力: ({C},) ベクトル', fontsize=12, ha='center',
           transform=ax.transAxes)
    ax.axis('off')
    
    # 出力
    ax = axes[2]
    gap_output = feature_maps.mean(axis=(1, 2))
    
    bars = ax.barh(range(C), gap_output, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax.set_yticks(range(C))
    ax.set_yticklabels([f'Ch {i}' for i in range(C)])
    ax.set_xlabel('平均値', fontsize=12)
    ax.set_title(f'GAP出力\n({C},) ベクトル', fontsize=14)
    ax.set_xlim(0, 1)
    
    for bar, val in zip(bars, gap_output):
        ax.text(val + 0.02, bar.get_y() + bar.get_height()/2, 
               f'{val:.3f}', va='center', fontsize=12)
    
    plt.suptitle('Global Average Pooling (GAP)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"入力形状: {feature_maps.shape}")
    print(f"出力形状: {gap_output.shape}")
    print(f"出力値: {gap_output}")

visualize_global_pooling()

In [ ]:
def compare_gap_vs_fc():
    """GAPと全結合層の比較"""
    print("="*60)
    print("GAP vs 全結合層 の比較")
    print("="*60)
    
    # 仮の設定
    channels = 512
    spatial = 7
    num_classes = 1000
    
    print(f"\n想定: {channels}チャンネル × {spatial}×{spatial} → {num_classes}クラス")
    
    # 全結合層のパラメータ数
    fc_params = channels * spatial * spatial * num_classes
    print(f"\n【全結合層】")
    print(f"  パラメータ数: {channels} × {spatial} × {spatial} × {num_classes} = {fc_params:,}")
    print(f"  入力サイズ固定: {spatial}×{spatial}でないと使えない")
    
    # GAP + 全結合のパラメータ数
    gap_params = channels * num_classes
    print(f"\n【GAP + 全結合層】")
    print(f"  パラメータ数: {channels} × {num_classes} = {gap_params:,}")
    print(f"  入力サイズ: 任意のH×Wに対応可能")
    
    print(f"\n【削減率】")
    print(f"  {fc_params / gap_params:.1f}倍のパラメータ削減！")
    print(f"  ({fc_params:,} → {gap_params:,})")

compare_gap_vs_fc()

### GAPの利点

1. **パラメータの大幅削減**: 上の例で49倍削減
2. **任意の入力サイズに対応**: 空間サイズに依存しない
3. **過学習の抑制**: パラメータが少ないため
4. **解釈可能性**: 各チャンネルの「クラス寄与度」として解釈可能

<a id="section5"></a>
## 5. ストライド畳み込み

プーリングの代わりに、**ストライド>1の畳み込み**でもダウンサンプリングできます。

In [ ]:
def visualize_strided_conv():
    """ストライド畳み込みの動作を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 入力
    input_size = 6
    ax = axes[0]
    
    for i in range(input_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # stride=2での畳み込み位置を表示
    colors = ['red', 'blue', 'green', 'orange']
    positions = [(0, 0), (0, 2), (2, 0), (2, 2)]
    
    for idx, (y, x) in enumerate(positions):
        rect = Rectangle((x, y), 3, 3,
                         linewidth=2, edgecolor=colors[idx], facecolor='none',
                         linestyle='--', alpha=0.8)
        ax.add_patch(rect)
    
    ax.set_xlim(-0.5, input_size + 0.5)
    ax.set_ylim(-0.5, input_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'入力 ({input_size}×{input_size})', fontsize=14)
    ax.invert_yaxis()
    ax.set_xlabel('3×3カーネル、stride=2の位置', fontsize=11)
    
    # 操作説明
    ax = axes[1]
    ax.text(0.5, 0.7, 'Strided Convolution', fontsize=18, ha='center', fontweight='bold',
           transform=ax.transAxes)
    ax.text(0.5, 0.5, 'kernel=3×3, stride=2', fontsize=14, ha='center',
           transform=ax.transAxes)
    ax.text(0.5, 0.3, 'padding=0 (valid)', fontsize=14, ha='center',
           transform=ax.transAxes)
    ax.text(0.5, 0.1, '出力サイズ: (6-3)/2 + 1 = 2', fontsize=12, ha='center',
           transform=ax.transAxes)
    ax.axis('off')
    
    # 出力
    ax = axes[2]
    output_size = 2
    
    for i in range(output_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 対応する色で出力を表示
    for idx, (y, x) in enumerate([(0, 0), (0, 1), (1, 0), (1, 1)]):
        rect = Rectangle((x, y), 1, 1,
                         linewidth=2, edgecolor=colors[idx], 
                         facecolor=colors[idx], alpha=0.3)
        ax.add_patch(rect)
    
    ax.set_xlim(-0.5, output_size + 0.5)
    ax.set_ylim(-0.5, output_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'出力 ({output_size}×{output_size})', fontsize=14)
    ax.invert_yaxis()
    
    plt.suptitle('ストライド畳み込みによるダウンサンプリング', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_strided_conv()

In [ ]:
def compare_strided_conv_with_pooling():
    """ストライド畳み込みとプーリングの出力比較"""
    # テスト画像
    np.random.seed(42)
    img = np.random.rand(1, 1, 8, 8).astype(np.float32)
    x = torch.tensor(img)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 入力
    axes[0, 0].imshow(img[0, 0], cmap='viridis', vmin=0, vmax=1)
    axes[0, 0].set_title('入力 (8×8)', fontsize=12)
    axes[0, 0].axis('off')
    
    # Max Pooling 2x2
    max_pool = nn.MaxPool2d(2, stride=2)
    max_out = max_pool(x).squeeze().numpy()
    axes[0, 1].imshow(max_out, cmap='viridis', vmin=0, vmax=1)
    axes[0, 1].set_title('Max Pool 2×2\n(4×4)', fontsize=12)
    axes[0, 1].axis('off')
    
    # Average Pooling 2x2
    avg_pool = nn.AvgPool2d(2, stride=2)
    avg_out = avg_pool(x).squeeze().numpy()
    axes[0, 2].imshow(avg_out, cmap='viridis', vmin=0, vmax=1)
    axes[0, 2].set_title('Avg Pool 2×2\n(4×4)', fontsize=12)
    axes[0, 2].axis('off')
    
    # 空白
    axes[1, 0].axis('off')
    
    # Strided Conv (ランダム重み)
    torch.manual_seed(42)
    strided_conv = nn.Conv2d(1, 1, 3, stride=2, padding=1)
    with torch.no_grad():
        strided_out = strided_conv(x).squeeze().numpy()
    axes[1, 1].imshow(strided_out, cmap='viridis')
    axes[1, 1].set_title('Strided Conv 3×3, s=2\n(4×4, ランダム重み)', fontsize=12)
    axes[1, 1].axis('off')
    
    # Strided Conv (平均重み = Average Poolingに近い)
    avg_conv = nn.Conv2d(1, 1, 2, stride=2, padding=0, bias=False)
    with torch.no_grad():
        avg_conv.weight.fill_(0.25)  # 1/4
        avg_conv_out = avg_conv(x).squeeze().numpy()
    axes[1, 2].imshow(avg_conv_out, cmap='viridis', vmin=0, vmax=1)
    axes[1, 2].set_title('Conv 2×2, s=2, weight=1/4\n(= Avg Pool)', fontsize=12)
    axes[1, 2].axis('off')
    
    plt.suptitle('プーリング vs ストライド畳み込み', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 数値比較
    print("Average Poolingと均一重み畳み込みの差（L2ノルム）:")
    print(f"  {np.linalg.norm(avg_out - avg_conv_out):.10f}")
    print("  → 実質的に同じ操作！")

compare_strided_conv_with_pooling()

<a id="section6"></a>
## 6. Pooling vs Strided Convolution

### 比較表

In [ ]:
def print_comparison_table():
    """プーリングとストライド畳み込みの比較表"""
    print("="*80)
    print("Pooling vs Strided Convolution の比較")
    print("="*80)
    
    comparisons = [
        ('学習パラメータ', 'なし', 'あり（カーネル重み）'),
        ('計算コスト', '低い', '中程度'),
        ('表現力', '固定（max/avg）', '学習により柔軟'),
        ('チェッカーボード問題', 'なし', '起こりうる'),
        ('勾配の流れ', 'sparse(max)/dense(avg)', 'dense'),
        ('位置情報の保持', '弱い', '比較的強い'),
        ('典型的な使用場所', '古典的CNN', 'ResNet以降'),
    ]
    
    print(f"\n{'観点':<25} {'Pooling':<25} {'Strided Conv':<25}")
    print("-"*75)
    for aspect, pool, strided in comparisons:
        print(f"{aspect:<25} {pool:<25} {strided:<25}")

print_comparison_table()

### チェッカーボード問題

ストライド畳み込みで発生しうるアーティファクト。

In [ ]:
def visualize_checkerboard_artifact():
    """チェッカーボードアーティファクトのデモ"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 転置畳み込み（アップサンプリング）でより顕著
    # ストライド畳み込みでも発生可能性あり
    
    # 均一入力
    x = torch.ones(1, 1, 4, 4)
    
    axes[0].imshow(x[0, 0].numpy(), cmap='gray', vmin=0, vmax=2)
    axes[0].set_title('入力（均一値）', fontsize=12)
    axes[0].axis('off')
    
    # 転置畳み込み（アップサンプリング）
    # ストライド2, カーネル3（奇数カーネルは重なりが不均一になりうる）
    deconv = nn.ConvTranspose2d(1, 1, 3, stride=2, padding=0, bias=False)
    with torch.no_grad():
        deconv.weight.fill_(1.0)
    
    output = deconv(x).detach()
    
    axes[1].imshow(output[0, 0].numpy(), cmap='gray')
    axes[1].set_title('転置畳み込み出力\n(チェッカーボード)', fontsize=12)
    axes[1].axis('off')
    
    # 解決策：カーネルサイズをストライドの倍数に
    deconv_good = nn.ConvTranspose2d(1, 1, 4, stride=2, padding=1, bias=False)
    with torch.no_grad():
        deconv_good.weight.fill_(1.0)
    
    output_good = deconv_good(x).detach()
    
    axes[2].imshow(output_good[0, 0].numpy(), cmap='gray')
    axes[2].set_title('カーネル4×4\n(均一出力)', fontsize=12)
    axes[2].axis('off')
    
    plt.suptitle('チェッカーボードアーティファクト（転置畳み込みで顕著）', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("解決策:")
    print("1. カーネルサイズをストライドの倍数にする")
    print("2. bilinear/nearest補間 + 通常畳み込みを使う")

visualize_checkerboard_artifact()

### 現代のアーキテクチャでの使い分け

In [ ]:
def show_architecture_choices():
    """各アーキテクチャでのダウンサンプリング戦略"""
    architectures = [
        ('VGGNet (2014)', 'Max Pooling 2×2', '古典的アプローチ'),
        ('ResNet (2015)', 'Strided Conv 3×3, s=2', '学習可能なダウンサンプリング'),
        ('DenseNet (2016)', 'Avg Pool + Conv 1×1', 'Transition層で使用'),
        ('MobileNet (2017)', 'Depthwise Conv s=2', '効率重視'),
        ('EfficientNet (2019)', 'Strided Conv', 'NASで最適化'),
        ('Vision Transformer (2020)', 'Patch Embedding', '畳み込みでパッチ化'),
        ('ConvNeXt (2022)', 'Strided Conv 4×4, s=4', 'Transformerスタイル'),
    ]
    
    print("="*80)
    print("各アーキテクチャのダウンサンプリング戦略")
    print("="*80)
    print(f"\n{'アーキテクチャ':<25} {'ダウンサンプリング':<25} {'備考':<25}")
    print("-"*75)
    for arch, ds, note in architectures:
        print(f"{arch:<25} {ds:<25} {note:<25}")
    
    print("\n結論: 最近のアーキテクチャではストライド畳み込みが主流")

show_architecture_choices()

<a id="summary"></a>
## 7. まとめ

### 学んだこと

1. **ダウンサンプリングの目的**
   - 計算量削減
   - 受容野の急速な拡大
   - 並進不変性の強化

2. **プーリングの種類**
   - **Max Pooling**: 最も顕著な特徴を保持、中間層で使用
   - **Average Pooling**: 全体の傾向を保持、ノイズに頑健
   - **Global Pooling**: 空間次元を完全に集約、最終層で使用

3. **ストライド畳み込み**
   - 学習可能なダウンサンプリング
   - より柔軟な表現が可能
   - 現代のアーキテクチャで主流

4. **使い分けの指針**
   - 計算効率重視 → プーリング
   - 表現力重視 → ストライド畳み込み
   - 最終層 → Global Average Pooling

### 次のノートブック

次のノートブックでは、受容野と3D Gaussian Splatting (3DGS)のアナロジーを探ります。

- CNNの受容野とGaussianの影響範囲の類似性
- 空間的な情報処理の統一的理解
- NeoVerseプロジェクトとの関連